In [1]:
import tensorflow as tf
from tensorflow.keras import datasets

# 1. Load CIFAR-10 Dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# 2. Normalize pixel values to be between 0 and 1
# ANN ke liye normalization bohat zaroori hai taake gradients stable rahein
train_images, test_images = train_images / 255.0, test_images / 255.0

# 3. Class names for reference
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Training data shape: {train_images.shape}")
print(f"Testing data shape: {test_images.shape}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
Training data shape: (50000, 32, 32, 3)
Testing data shape: (10000, 32, 32, 3)


In [2]:
# Data Split and Validation Setup
from sklearn.model_selection import train_test_split


x_train_final, x_val, y_train_final, y_val = train_test_split(
    train_images,
    train_labels,
    test_size=0.2,
    random_state=42
)

print(f"Total Training Images: {len(x_train_final)}")
print(f"Total Validation Images: {len(x_val)}")
print(f"Total Test Images (Unseen): {len(test_images)}")

Total Training Images: 40000
Total Validation Images: 10000
Total Test Images (Unseen): 10000


In [3]:
# 1. import libraries
import tensorflow as tf
from tensorflow.keras import layers, models

# 2. ANN Model build
def build_ann_baseline():
    model = models.Sequential([
        # convert Image 3D (32,32,3) to 1D
        layers.Flatten(input_shape=(32, 32, 3)),

        # Hidden Layers by using Relu
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),

        # Output Layer (for 10 classes)
        layers.Dense(10, activation='softmax')
    ])

    # Model compile
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 3. Model initialization
ann_baseline = build_ann_baseline()

# 4. Model trainning
history_ann_a = ann_baseline.fit(
    x_train_final,
    y_train_final,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, y_val)
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.2705 - loss: 2.0186 - val_accuracy: 0.3620 - val_loss: 1.7528
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3803 - loss: 1.7292 - val_accuracy: 0.3901 - val_loss: 1.7242
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4239 - loss: 1.6124 - val_accuracy: 0.4336 - val_loss: 1.5847
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4509 - loss: 1.5520 - val_accuracy: 0.4423 - val_loss: 1.5521
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4606 - loss: 1.5092 - val_accuracy: 0.4445 - val_loss: 1.5542
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4745 - loss: 1.4697 - val_accuracy: 0.4520 - val_loss: 1.5412
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4824 - loss: 1.4404 - val_accuracy: 0.4581 - val_loss: 1.5257
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4973 - loss: 1.4092 - val_accuracy: 0.

In [4]:
import os

# ANN  folders making
os.makedirs('ann_results/models', exist_ok=True)
os.makedirs('ann_results/reports', exist_ok=True)

print("ANN folders 'models' and 'reports' are ready!")

ANN folders 'models' and 'reports' are ready!


In [5]:
# Scenario A model save
ann_baseline.save('ann_results/models/ann_scenario_a_baseline.keras')
print("Scenario A Model is saved!")

Scenario A Model is saved!


In [6]:
# Training results
with open('ann_results/reports/ann_scenario_a_report.txt', 'w') as f:
    f.write("ANN Scenario A: Baseline Report\n")
    f.write("===============================\n")
    f.write(f"Final Training Accuracy: {history_ann_a.history['accuracy'][-1]:.4f}\n")
    f.write(f"Final Validation Accuracy: {history_ann_a.history['val_accuracy'][-1]:.4f}\n")
    f.write(f"Final Training Loss: {history_ann_a.history['loss'][-1]:.4f}\n")
    f.write(f"Final Validation Loss: {history_ann_a.history['val_loss'][-1]:.4f}\n")

print("Scenario A Report is saved!")

Scenario A Report is saved!


In [7]:
# Classification Report & Confusion Matrix Code
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 1. Get model predictions on the validation set
# Predict probability for each class
y_pred_probs = ann_baseline.predict(x_val)

# Convert probabilities to class labels (0 to 9)
y_pred = np.argmax(y_pred_probs, axis=1)

# 2. Generate the Classification Report
# This gives Precision, Recall, and F1-Score for every class
report = classification_report(y_val, y_pred, target_names=class_names)

print("ANN Scenario A: Classification Report")
print("======================================")
print(report)

# 3. Save the report to a text file
with open('ann_results/reports/ann_scenario_a_classification_report.txt', 'w') as f:
    f.write("ANN Scenario A: Detailed Classification Report\n")
    f.write(report)

print("\nReport saved to 'ann_results/reports/'")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
ANN Scenario A: Classification Report
              precision    recall  f1-score   support

    airplane       0.54      0.54      0.54       973
  automobile       0.61      0.58      0.60       979
        bird       0.41      0.37      0.39      1030
         cat       0.34      0.44      0.38      1023
        deer       0.47      0.36      0.41       933
         dog       0.40      0.34      0.37      1015
        frog       0.51      0.54      0.53       996
       horse       0.55      0.54      0.55       994
        ship       0.50      0.73      0.59      1017
       truck       0.62      0.46      0.53      1040

    accuracy                           0.49     10000
   macro avg       0.50      0.49      0.49     10000
weighted avg       0.50      0.49      0.49     10000


Report saved to 'ann_results/reports/'


In [11]:
# ANN Scenario B: Optimized Learning Rate
from tensorflow.keras import optimizers

def build_ann_scenario_b():
    model = models.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    # Learning rate 0.0001 (Default se 10x kam)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

ann_model_b = build_ann_scenario_b()
history_ann_b = ann_model_b.fit(x_train_final, y_train_final, epochs=10, batch_size=64, validation_data=(x_val, y_val))

# Save for Scenario B
ann_model_b.save('ann_results/models/ann_scenario_b_tuned.keras')

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.2855 - loss: 1.9948 - val_accuracy: 0.3756 - val_loss: 1.7532
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3907 - loss: 1.7149 - val_accuracy: 0.4185 - val_loss: 1.6422
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4251 - loss: 1.6289 - val_accuracy: 0.4330 - val_loss: 1.5875
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4498 - loss: 1.5537 - val_accuracy: 0.4556 - val_loss: 1.5381
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4701 - loss: 1.5049 - val_accuracy: 0.4606 - val_loss: 1.5133
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4800 - loss: 1.4637 - val_accuracy: 0.4683 - val_loss: 1.4913
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4958 - loss: 1.4358 - val_accuracy: 0.4746 - val_loss: 1.4738
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5058 - loss: 1.3897 - val_accuracy: 0.

In [12]:
# ANN Scenario C: Dropout + Augmentation
from tensorflow.keras import layers, models

def build_ann_scenario_c():
    model = models.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),  # Adding Dropout
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

ann_model_c = build_ann_scenario_c()

# Data Augmentation (Wahi jo CNN mein use ki thi)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15, horizontal_flip=True)

print("Starting Optimized ANN Training...")
history_ann_c = ann_model_c.fit(datagen.flow(x_train_final, y_train_final, batch_size=64),
                                 epochs=10, validation_data=(x_val, y_val))

# Save for Scenario C
ann_model_c.save('ann_results/models/ann_scenario_c_optimized.keras')

Starting Optimized ANN Training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.1863 - loss: 2.1887 - val_accuracy: 0.2844 - val_loss: 1.9442
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.2726 - loss: 1.9609 - val_accuracy: 0.3206 - val_loss: 1.8480
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.2924 - loss: 1.9138 - val_accuracy: 0.3482 - val_loss: 1.8208
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.3053 - loss: 1.8813 - val_accuracy: 0.3430 - val_loss: 1.8240
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.3224 - loss: 1.8529 - val_accuracy: 0.3597 - val_loss: 1.7909
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.3299 - loss: 1.8402 - val_accuracy: 0.3716 - val_loss: 1.7448
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.3303 - loss: 1.8167 - val_accuracy: 0.3752 - val_loss: 1.7805
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.3295 - loss: 1.8259 - 

In [13]:
import pickle

# --- SAVE SCENARIO A ---
ann_baseline.save('ann_results/models/ann_scenario_a_baseline.keras')
with open('ann_results/reports/history_ann_a.pkl', 'wb') as f:
    pickle.dump(history_ann_a.history, f)

# --- SAVE SCENARIO B ---
ann_model_b.save('ann_results/models/ann_scenario_b_tuned.keras')
with open('ann_results/reports/history_ann_b.pkl', 'wb') as f:
    pickle.dump(history_ann_b.history, f)

# --- SAVE SCENARIO C ---
ann_model_c.save('ann_results/models/ann_scenario_c_optimized.keras')
with open('ann_results/reports/history_ann_c.pkl', 'wb') as f:
    pickle.dump(history_ann_c.history, f)

print("Success: Scenarios A, B, and C (Models & History) are all saved!")

Success: Scenarios A, B, and C (Models & History) are all saved!


In [15]:
import os
import pickle
import matplotlib.pyplot as plt

# 1. Define the path and create folders first
base_path = 'ann_outputs'
os.makedirs(f'{base_path}/models', exist_ok=True)
os.makedirs(f'{base_path}/histories', exist_ok=True)
os.makedirs(f'{base_path}/plots', exist_ok=True)

# 2. Define the plotting function (now it knows what base_path is)
def save_final_plot(history_obj, name):
    plt.figure(figsize=(10, 4))
    plt.plot(history_obj.history['accuracy'], label='Train Acc')
    plt.plot(history_obj.history['val_accuracy'], label='Val Acc')
    plt.title(f'Results for {name}')
    plt.legend()
    plt.savefig(f'{base_path}/plots/{name}_plot.png')
    plt.close()

# 3. List of scenarios to save
scenarios = [
    ('ann_baseline', 'history_ann_a', 'scenario_a'),
    ('ann_model_b', 'history_ann_b', 'scenario_b'),
    ('ann_model_c', 'history_ann_c', 'scenario_c')
]

print("--- Starting the Saving Process ---")

# 4. Saving loop
for model_var, hist_var, name in scenarios:
    # Checking if the variables exist in Colab memory
    if model_var in locals() and hist_var in locals():
        model_obj = locals()[model_var]
        hist_obj = locals()[hist_var]

        # Save Model file
        model_obj.save(f'{base_path}/models/{name}_model.keras')

        # Save History Pickle file
        with open(f'{base_path}/histories/{name}_history.pkl', 'wb') as f:
            pickle.dump(hist_obj.history, f)

        # Save the Plot
        save_final_plot(hist_obj, name)

        print(f"✅ {name.upper()} saved successfully.")
    else:
        print(f"⚠️ {name.upper()} variables not found. Skipping...")

print("\nAll available work is now saved in the 'ann_outputs' folder.")

--- Starting the Saving Process ---
✅ SCENARIO_A saved successfully.
✅ SCENARIO_B saved successfully.
✅ SCENARIO_C saved successfully.

All available work is now saved in the 'ann_outputs' folder.


In [16]:
import shutil

# Zip the 'ann_outputs' folder into a single file
shutil.make_archive('ann_final_submission', 'zip', 'ann_outputs')

print("🚀 'ann_final_submission.zip' is ready!")

🚀 'ann_final_submission.zip' is ready!
